In [1]:
import os
import requests
import json


In [85]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/46.0.2490.80'
}

def fetch_tradingview_data(url):
    payload = {}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    print(response)
    # return response
    if response.status_code == 200:
        data = response.json()
        return data['data']['ideas']['data']['items']
    else:
        print(f"Error: Unable to fetch data. Status code {response.status_code}")
        return []

def save_chart_image(image_url, save_path):
    image_response = requests.get(image_url, stream=True)
    if image_response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in image_response.iter_content(1024):
                file.write(chunk)
    else:
        print(f"Error: Unable to download image from {image_url}. Status code {image_response.status_code}")

def extract_and_save_data(items, output_dir,fileName="page1"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    ids = []

    json_data_to_save = []

    for item in items:
        id = item.get("id")
        if(id in ids):
            continue
        
        ids.append(id)
        # Extract useful data
        data = {
            "id": item.get("id"),
            "name": item.get("name"),
            "description": item.get("description"),
            "created_at": item.get("created_at"),
            "chart_url": item.get("chart_url"),
            "symbol": item.get("symbol", {}).get("name"),
            "user": {
                "id": item.get("user", {}).get("id"),
                "username": item.get("user", {}).get("username"),
                "pro_plan": item.get("user", {}).get("pro_plan"),
            },
            "likes_count": item.get("likes_count"),
        }

       

        # Save the chart image
        # image_url = item.get("image", {}).get("big")
        # if image_url:
        #     image_path = os.path.join(output_dir, f"{data['id']}.png")
        #     save_chart_image(image_url, image_path)

        json_data_to_save.append(data)

         # Save the data as JSON
    json_path = os.path.join(output_dir, f"{fileName}.json")
    with open(json_path, 'w') as json_file:
        json.dump(json_data_to_save, json_file, indent=4)

    print(len(ids))


In [79]:
all_items1 = []
for i in range(3,51):
    url = f'https://www.tradingview.com/ideas/page-{i}/?component-data-only=1'
    print(f'fetching {i} {url} ')
    all_items1 += fetch_tradingview_data(url)

fetching 3 https://www.tradingview.com/ideas/page-3/?component-data-only=1 
<Response [200]>
fetching 4 https://www.tradingview.com/ideas/page-4/?component-data-only=1 
<Response [200]>
fetching 5 https://www.tradingview.com/ideas/page-5/?component-data-only=1 
<Response [200]>
fetching 6 https://www.tradingview.com/ideas/page-6/?component-data-only=1 
<Response [200]>
fetching 7 https://www.tradingview.com/ideas/page-7/?component-data-only=1 
<Response [200]>
fetching 8 https://www.tradingview.com/ideas/page-8/?component-data-only=1 
<Response [200]>
fetching 9 https://www.tradingview.com/ideas/page-9/?component-data-only=1 
<Response [200]>
fetching 10 https://www.tradingview.com/ideas/page-10/?component-data-only=1 
<Response [200]>
fetching 11 https://www.tradingview.com/ideas/page-11/?component-data-only=1 
<Response [200]>
fetching 12 https://www.tradingview.com/ideas/page-12/?component-data-only=1 
<Response [200]>
fetching 13 https://www.tradingview.com/ideas/page-13/?component

In [81]:
len(items+all_items1)

1384

In [82]:
if not os.path.exists("tradingview_data"):
        os.makedirs("tradingview_data")
json_path = os.path.join("tradingview_data", f"full_unProcessed.json")
with open(json_path, 'w') as json_file:
    json.dump(items+all_items1, json_file, indent=4)

In [49]:
url = "https://www.tradingview.com/ideas/page-3"
output_directory = "tradingview_data"

# Fetch and process data
items = fetch_tradingview_data(url)
extract_and_save_data(items, output_directory,"page2")
print(f"Data extraction and image download complete. Saved in '{output_directory}' folder.")

<Response [200]>
Data extraction and image download complete. Saved in 'tradingview_data' folder.


In [86]:
extract_and_save_data(items+all_items1, output_directory,"pageUpto50")
print(f"Data extraction and image download complete. Saved in '{output_directory}' folder.")

941
Data extraction and image download complete. Saved in 'tradingview_data' folder.


In [46]:
a = items.json()['data']['ideas']['data']['items'][0:10]
extract_and_save_data(a, output_directory)

In [122]:
import os
import json
import numpy as np
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.applications import ResNet50, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
# Directories
data_dir = "tradingview_data"
processed_text_dir = "processed_text"
processed_images_dir = "processed_images"

In [116]:
a = np.array([.3])
b = np.array([.3])
a*b

array([0.09])

In [123]:
def process_image_data():
    """Load and preprocess images."""
    model = ResNet50(weights="imagenet", include_top=False, pooling="avg")
    d = []

    for file_name in os.listdir(data_dir):
        if file_name.endswith(".png"):
            a = {}
            file_path = os.path.join(data_dir, file_name)
            image = Image.open(file_path).convert("RGB")
            image_array = np.array(image)
            image_array = np.expand_dims(image_array, axis=0)
            image_array = preprocess_input(image_array)

            # Extract features using ResNet50
            features = model.predict(image_array)
            # np.save(os.path.join(processed_images_dir, f"{os.path.splitext(file_name)[0]}.npy"), features)
            id = file_name.split(".")[0]

            a['id'] = id
            a['npy'] = features

            d.append(a)
            
    return d


In [124]:
process_data = process_image_data()

1/1 [==============================] - 1s 1s/step


In [141]:
json_path = os.path.join("tradingview_data", f"pageUpto50.json")
with open(json_path, 'r') as json_file:
    data = json.load(json_file)

In [146]:

def get_image_npy(id):
    for img_obj in process_data:
        if(img_obj['id']==id):
            return img_obj['npy']
    return np.array([])

In [157]:
if not np.array([]):
    print("ff")

ff


In [158]:
for i,item in enumerate(data):
    print(i,item['id'])
    img = get_image_npy(f"{item['id']}")
    if(not img.shape[0]):
        print("oops.")
        continue
    item['image_py'] = img

0 18115109
1 18107426
2 18115156
3 18117413
4 18114836
5 18116207
6 18114895
7 18115309
8 18116296
9 18116704
10 18109442
11 18115939
12 18116497
13 18118310
14 18115809
15 18109444
16 18112419
17 18118269
18 18115663
19 18114506
20 18113363
21 18113959
22 18113901
23 18117012
24 18112179
25 18114800
26 18116803
27 18114926
28 18114616
29 18115807
30 18114727
31 18116629
32 18114564
33 18114645
34 18116768
35 18117874
36 18116563
37 18114715
38 18114580
39 18115024
40 18114853
41 18114686
42 18100213
43 18114959
44 18116299
45 18117286
46 18114838
47 18112693
48 18084949
49 18118249
50 18116243
51 18114512
52 18114723
53 18114725
54 18117273
55 18114970
56 18096779
57 18117655
58 18116295
59 18114927
60 18114461
61 18117370
62 18114712
63 18114943
64 18117640
65 18117406
66 18116075
67 18114648
68 18117627
69 18117613
70 18114683
71 18114673
72 18117647
73 18117623
74 18117553
75 18117401
76 18114741
77 18117473
78 18114719
79 18114631
80 18117338
81 18114746
82 18117355
83 18114653
84

In [159]:
data[0]

{'id': 18115109,
 'name': '“Dynamic Liquidity Analyzer” ',
 'description': 'Trendlines\nThe white lines are the markers of order amidst chaos. Solid and dashed, they connect critical highs and lows, serving as pathways that reveal the future of price action. They outline the potential for breakouts or breakdowns—decision points where the market bows to our strategy. \nThese lines carve the battlefield, where traders either rise or fall. These highs and lows are structured almost uniquely, deviating from the conventional setup of ordinary highs and lows.\n\nVolume Weighted Average Price (VWAP)\nThe blue stair-step line is no ordinary indicator—it is the foundation of fair value. This VWAP dynamically calculates the average price weighted by volume, serving as a stronghold for price action. The market respects it as a key support and resistance level. A bounce off this VWAP is a signal of bullish resurgence, while a break below hints at bearish momentum. It’s not just a line—it’s the axi

In [161]:
import pickle
output_file = os.path.join("tradingview_data", "data.pkl")
with open(output_file, "wb") as f:
    pickle.dump(data, f)

print(f"Data saved to {output_file}")

Data saved to tradingview_data\data.pkl


In [164]:
 import os
import json
import random
import shutil
import zipfile
import csv
# from datasets import Dataset, DatasetDict, load_dataset

# Paths
data_folder = "tradingview_data"
json_file = os.path.join(data_folder, "pageUpto50.json")
train_folder = os.path.join(data_folder, "train")
test_folder = os.path.join(data_folder, "test")
metadata_file = os.path.join(data_folder, "metadata.csv")

# Create folders if not exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Load JSON data
with open(json_file, "r") as f:
    ideas = json.load(f)

# Shuffle data and split into 80/20
random.shuffle(ideas)
split_index = int(0.8 * len(ideas))
train_ideas = ideas[:split_index]
test_ideas = ideas[split_index:]

# Function to move images
def move_images(ideas, target_folder):
    for idea in ideas:
        image_file = os.path.join(data_folder, f"{idea['id']}.png")
        if os.path.exists(image_file):
            shutil.move(image_file, target_folder)

# Move images to train and test folders
move_images(train_ideas, train_folder)
move_images(test_ideas, test_folder)

# Zip train and test folders
def zip_folder(folder, zip_name):
    zip_path = os.path.join(data_folder, zip_name)
    with zipfile.ZipFile(zip_path, "w") as zf:
        for root, _, files in os.walk(folder):
            for file in files:
                zf.write(os.path.join(root, file),
                         arcname=os.path.relpath(os.path.join(root, file), folder))

zip_folder(train_folder, "train.zip")
zip_folder(test_folder, "test.zip")

# Create metadata.csv
with open(metadata_file, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["file_name", "title","description","created_at","chart_url","symbol","likes_count"])
    for idea in ideas:
        file_name = f"{idea['id']}.png"
        title = f"{idea['name']}"
        description = f" {idea['description']}"
        created_at = f"{idea['created_at']}"
        chart_url = f"{idea['chart_url']}"
        symbol = f"{idea['symbol']}"
        likes_count = f"{idea['likes_count']}"
        writer.writerow([file_name, title,description,created_at,chart_url,symbol,likes_count ])

# Push to Hugging Face
# Ensure you have `datasets` library installed and authenticated Hugging Face CLI
# repo_id = "DiljitSingh14/tradingIdeas"
# train_zip_path = os.path.join(data_folder, "train.zip")
# test_zip_path = os.path.join(data_folder, "test.zip")

# # Add metadata and zip files to dataset repo
# dataset = load_dataset(repo_id)
# dataset.push_to_hub(repo_id, add_files={
#     "metadata.csv": metadata_file,
#     "train.zip": train_zip_path,
#     "test.zip": test_zip_path
# })

print("Data split, metadata created, and changes pushed to Hugging Face successfully.")


Data split, metadata created, and changes pushed to Hugging Face successfully.


In [167]:
!pip install datasets

     ---------------------------------------- 0.0/480.6 kB ? eta -:--:--
     ----------- -------------------------- 143.4/480.6 kB 4.3 MB/s eta 0:00:01
     -------------------------------------- 480.6/480.6 kB 6.0 MB/s eta 0:00:00
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
     ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
     - -------------------------------------- 1.2/25.1 MB 37.7 MB/s eta 0:00:01
     ----- ---------------------------------- 3.6/25.1 MB 46.2 MB/s eta 0:00:01
     ------- -------------------------------- 4.9/25.1 MB 39.1 MB/s eta 0:00:01
     ------------ --------------------------- 7.9/25.1 MB 46.0 MB/s eta 0:00:01
     ---------------- ---------------------- 10.9/25.1 MB 54.4 MB/s eta 0:00:01
     --------------------- ----------------- 14.0/25.1 MB 54.7 MB/s eta 0:00:01
     ------------------------- ------------- 16.5/25.1 MB 65.6 MB/s eta 0:00:01
     ---------------------------- ---------- 18.4/25.1 MB 59.5 MB/s eta 0:00:01
   

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optionlab 1.2.0 requires pydantic<3.0.0,>=2.6.3, which is not installed.
optionlab 1.2.0 requires scipy<2.0.0,>=1.12.0, but you have scipy 1.10.1 which is incompatible.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
